In [45]:
import os
import glob

import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_from_disk
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer



LABELS = ('fairness_and_equality', 'security_and_defense', 'crime_and_punishment', 'morality',
          'policy_prescription_and_evaluation', 'capacity_and_resources', 'economic', 'cultural_identity',
          'health_and_safety', 'quality_of_life', 'legality_constitutionality_and_jurisprudence',
          'political', 'public_opinion', 'external_regulation_and_reputation')

# Join Datasets from different languages

In [46]:
df_paths = glob.glob(os.path.join('..', '..', 'data', 'preprocessed', '*train_and_dev.csv'))

In [47]:
for i, df_path_i in enumerate(df_paths):
    df_i = pd.read_csv(df_path_i, index_col='id')
    df_i['language'] = os.path.basename(df_path_i).split('_')[1]

    if i == 0:
        df = df_i
    else:
        df = pd.concat([df, df_i])

In [48]:
df

,frames,raw_text,title,content,title_and_5_sentences,title_and_10_sentences,title_and_first_paragraph,title_and_first_sentence_each_paragraph,language
id,,,,,,,,,
24151,"Legality_Constitutionality_and_jurisprudence,H...",Байдену напомнили о его отношению к абортам в ...,Байдену напомнили о его отношению к абортам в ...,CNN напомнил о трансформации отношения Байдена...,Байдену напомнили о его отношению к абортам в ...,Байдену напомнили о его отношению к абортам в ...,Байдену напомнили о его отношению к абортам в ...,CNN напомнил о трансформации отношения Байдена...,ru
24150,"Legality_Constitutionality_and_jurisprudence,H...",В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,ru
24153,"Legality_Constitutionality_and_jurisprudence,F...","Байден пообещал американским мужчинам, что они...","Байден пообещал американским мужчинам, что они...",Решение Верховного суда об абортах ужасно. Не ...,"Байден пообещал американским мужчинам, что они...","Байден пообещал американским мужчинам, что они...","Байден пообещал американским мужчинам, что они...",Решение Верховного суда об абортах ужасно. «Эт...,ru
24152,"Legality_Constitutionality_and_jurisprudence,P...",Моё тело – моё дело: американки протестуют из-...,Моё тело – моё дело: американки протестуют из-...,Что случилось?\n\nВерховный суд США отменил ко...,Моё тело – моё дело: американки протестуют из-...,Моё тело – моё дело: американки протестуют из-...,Моё тело – моё дело: американки протестуют из-...,Что случилось? Верховный суд США отменил конст...,ru
24147,"Legality_Constitutionality_and_jurisprudence,E...",В США начались погромы из-за запрета абортов: ...,В США начались погромы из-за запрета абортов: ...,Почему скандальный закон приняли именно сейчас...,В США начались погромы из-за запрета абортов: ...,В США начались погромы из-за запрета абортов: ...,В США начались погромы из-за запрета абортов: ...,Почему скандальный закон приняли именно сейчас...,ru
...,...,...,...,...,...,...,...,...,...
25143,"Political,Security_and_defense","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Każdy, kto miał kiedyś nowy samochód wie, że p...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Każdy, kto miał kiedyś nowy samochód wie, że p...",po
2528,"Policy_prescription_and_evaluation,Economic,Ex...",Rosja może uniknąć bankructwa. Zachodnia machi...,Rosja może uniknąć bankructwa. Zachodnia machi...,"Nie jest pewne, czy obecne sankcje wystarczą d...",Rosja może uniknąć bankructwa. Zachodnia machi...,Rosja może uniknąć bankructwa. Zachodnia machi...,Rosja może uniknąć bankructwa. Zachodnia machi...,"Nie jest pewne, czy obecne sankcje wystarczą d...",po
2530,"Policy_prescription_and_evaluation,Economic,Ex...",Embargo na gaz i ropę z Rosji? Jednoznaczne st...,Embargo na gaz i ropę z Rosji? Jednoznaczne st...,"Marine Le Pen, rywalka Emmanuela Macrona w wyś...",Embargo na gaz i ropę z Rosji? Jednoznaczne st...,Embargo na gaz i ropę z Rosji? Jednoznaczne st...,Embargo na gaz i ropę z Rosji? Jednoznaczne st...,"Marine Le Pen, rywalka Emmanuela Macrona w wyś...",po


### Encode labels

In [54]:
mlb = MultiLabelBinarizer()
mlb.fit([LABELS])

labels_npy = mlb.transform(df.frames.str.lower().str.split(',')).astype(float)
df['label'] = [list(labels_npy[i, :]) for i in range(labels_npy.shape[0])]

### Do iterative stratification to create a train and holdout set, stratifying per language

In [55]:
splits = 3
mskf = MultilabelStratifiedKFold(n_splits=splits, shuffle=True, random_state=0)

train_dfs = []
test_dfs = []
for language, df_ in df.groupby('language'):
    X = df_[[col for col in df.columns if col not in ['label', 'frames']]]
    y = df_[[col for col in df.columns if col in ['label', 'frames']]]

    for train_index, test_index in mskf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        train_dfs.append(X_train.join(y_train))
        test_dfs.append(X_test.join(y_test))

        break

train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)

### Create Dataset object

In [56]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

dataset = DatasetDict({'train': train_ds, 'test': test_ds})

In [57]:
dataset

DatasetDict({
    train: Dataset({
        features: ['raw_text', 'title', 'content', 'title_and_5_sentences', 'title_and_10_sentences', 'title_and_first_paragraph', 'title_and_first_sentence_each_paragraph', 'language', 'frames', 'label', 'id'],
        num_rows: 1059
    })
    test: Dataset({
        features: ['raw_text', 'title', 'content', 'title_and_5_sentences', 'title_and_10_sentences', 'title_and_first_paragraph', 'title_and_first_sentence_each_paragraph', 'language', 'frames', 'label', 'id'],
        num_rows: 530
    })
})

#### Save it

In [58]:
dataset.save_to_disk(os.path.join('..', '..', 'data', 'preprocessed','multilingual_train_test_ds.hf'))

Saving the dataset (0/1 shards):   0%|          | 0/1059 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/530 [00:00<?, ? examples/s]

# Test a model

### Load datset

In [11]:
id2label= {idx:label for idx, label in enumerate(mlb.classes_)}
label2id = {label:idx for idx, label in enumerate(mlb.classes_)}

In [12]:
dataset = load_from_disk(os.path.join('..', '..', 'data', 'preprocessed','multilingual_train_test_ds.hf'))

In [13]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

In [18]:
dataset['train']

Dataset({
    features: ['raw_text', 'title', 'content', 'title_and_5_sentences', 'title_and_10_sentences', 'title_and_first_paragraph', 'title_and_first_sentence_each_paragraph', 'language', 'frames', 'label', 'id'],
    num_rows: 1059
})

In [59]:
def preprocess_data(examples, unit_of_analysis):
  # take a batch of texts
  text = examples[unit_of_analysis]

  # encode them
  encoding = tokenizer(text, truncation=True)

  # Add their respective labels
  encoding["labels"] = examples['label']

  return encoding

In [21]:
encoded_dataset = dataset.map(lambda ex: preprocess_data(ex, 'raw_text'), batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [29]:
example = encoded_dataset['train'][0]

In [60]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-large",
    problem_type="multi_label_classification",
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [39]:
from transformers import TrainingArguments, Trainer
import evaluate

batch_size = 8
metric_name = "f1-micro"


args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [41]:

import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average = 'micro')

    # return as dictionary
    metrics = {
        'f1': f1_micro_average,
        'precision': precision,
        'roc_auc': roc_auc,
        'accuracy': accuracy
    }

    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    return result

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
trainer.train()

/home/juanbermeo/anaconda3/envs/Framing_HF/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1059
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 665
  Number of trainable parameters = 559904782
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.82 GiB total capacity; 3.10 GiB already allocated; 7.44 MiB free; 3.10 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [14]:
classifier = pipeline(model='xlm-roberta-large', task='zero-shot-classification')

KeyboardInterrupt: 

In [ ]:
classifier = pipeline(model="joeddav/xlm-roberta-large-xnli", task="zero-shot-classification")

In [ ]:
classifier = transformers.pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")